In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import torch
import wandb

from config import Environment, TrainConfig
from denoising.train import prepare_training
from denoising.utils import seed_everything

In [3]:
CWD = Path.cwd()
env = Environment(_env_file=CWD / '../env')
wandb.login(key=env.wandb_api_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/d.nesterov/.netrc
wandb: Currently logged in as: dmitrylala to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Конфиг обучения и подготовка

In [15]:
run_idx = 28
save_dir = CWD / 'mri'


cfg = TrainConfig(
    # Datasets params
    train_dset='mri_pm_train',
    test_dset='mri_pm_test',
    train_batch_size=64,
    test_batch_size=128,
    # Model params
    name_model='mri-hno-run28',
    cfg_fno={
        'n_modes': (8, 8),
        'in_channels': 1,
        'hidden_channels': 32,
        'lifting_channel_ratio': 6,
        'projection_channel_ratio': 32,
        'out_channels': 1,
        'factorization': 'dense',
        'n_layers': 4,
        'rank': 0.42,
        'spectral': 'hartley',
    },
    # Run params
    random_seed=42,
    device='cuda:2',
    run_name=f'Run {run_idx}, HNO',
    save_weights_path=save_dir / f'run-{run_idx}-weights.pt',
    # Train params
    n_epochs=20,
    lr=1e-3,
    wandb_log=True,
    save_dir=save_dir / f'run-{run_idx}',
    verbose=True,
)

In [16]:
trainer, train_kwargs, run = prepare_training(env, cfg)

Got n_samples = 8380  in dataset mri_pm_train        with sample size = torch.Size([1, 145, 145])
Got n_samples = 2093  in dataset mri_pm_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 2093  in dataset mri_gt_test         with sample size = torch.Size([1, 145, 145])
Got n_samples = 137   in dataset bsd_synth_0.01_train with sample size = torch.Size([1, 321, 481])
Got n_samples = 77    in dataset bsd_synth_0.01_test with sample size = torch.Size([1, 321, 481])
Got n_samples = 12296 in dataset sidd_train          with sample size = torch.Size([3, 512, 512])
Got n_samples = 3008  in dataset sidd_test           with sample size = torch.Size([3, 512, 512])
torch.Size([64, 1, 145, 145]) torch.Size([64, 1, 145, 145])
Loaded  model mri-fno-neuralop with n_parameters = 2010449
Loaded  model mri-fno-custom   with n_parameters = 2010449
Loaded  model mri-fno-dense    with n_parameters = 4476513
Loaded  model mri-hno-separable with n_parameters = 2248289
Loaded  model mr

Logging to wandb enabled: True


# Запуск

In [17]:
seed_everything(cfg.random_seed)
trainer.train(**train_kwargs)

Training on 8380 samples
Testing on [2093] samples         on resolutions ['test'].
Raw outputs of shape torch.Size([64, 1, 145, 145])
[0] time=52.93, avg_loss=0.1995, train_err=12.7597
Eval: test_h1=0.1186, test_l2=0.0441
Saved training state to /home/d.nesterov/denoising-fno/notebooks/mri/run-27
[1] time=51.52, avg_loss=0.1143, train_err=7.3087
Eval: test_h1=0.1106, test_l2=0.0413
Saved training state to /home/d.nesterov/denoising-fno/notebooks/mri/run-27
[2] time=52.03, avg_loss=0.1060, train_err=6.7823
Eval: test_h1=0.1034, test_l2=0.0382
Saved training state to /home/d.nesterov/denoising-fno/notebooks/mri/run-27
[3] time=51.59, avg_loss=0.1023, train_err=6.5466
Eval: test_h1=0.1013, test_l2=0.0366
Saved training state to /home/d.nesterov/denoising-fno/notebooks/mri/run-27
[4] time=51.43, avg_loss=0.1004, train_err=6.4197
Eval: test_h1=0.0995, test_l2=0.0442
Saved training state to /home/d.nesterov/denoising-fno/notebooks/mri/run-27
[5] time=51.36, avg_loss=0.0985, train_err=6.2983

{'train_err': 5.568957874793132,
 'avg_loss': 0.08705650138399766,
 'epoch_train_time': 51.7722720708698,
 'test_h1': tensor(0.0875, device='cuda:2'),
 'test_l2': tensor(0.0372, device='cuda:2'),
 'test_outputs': <wandb.sdk.data_types.image.Image at 0x7fd1a4257a60>}

In [18]:
if run is not None:
    run.finish()

avg_loss,█▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
lr,████▇▇▇▆▆▆▅▅▄▄▃▃▂▂▁▁
test_h1,█▆▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁
test_l2,▄▃▂▁▄▅▅▆▇█▇▇▅▄▃▃▂▂▂▁
time,█▂▄▂▂▁▁▁▁▃▅▂▅▅▄▅▅▇▅▃
train_err,█▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
avg_loss,0.08706
lr,0.00025
test_h1,0.08754
test_l2,0.03723
time,51.77227


In [19]:
torch.save(trainer.model.to('cpu').state_dict(), cfg.save_weights_path)
print(f'Saved to {cfg.save_weights_path}')

Saved to /home/d.nesterov/denoising-fno/notebooks/mri/run-27-weights.pt
